In [86]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [87]:
df = pd.read_csv("supermarket_sales.csv")
df.head(5)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Invoice ID               1000 non-null   object 
 1   Branch                   1000 non-null   object 
 2   City                     1000 non-null   object 
 3   Customer type            1000 non-null   object 
 4   Gender                   1000 non-null   object 
 5   Product line             1000 non-null   object 
 6   Unit price               1000 non-null   float64
 7   Quantity                 1000 non-null   int64  
 8   Tax 5%                   1000 non-null   float64
 9   Total                    1000 non-null   float64
 10  Date                     1000 non-null   object 
 11  Time                     1000 non-null   object 
 12  Payment                  1000 non-null   object 
 13  cogs                     1000 non-null   float64
 14  gross margin percentage  

In [89]:
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
df['Time'] = pd.to_datetime(df['Time'])
# df['Time'] = pd.to_datetime(df['Time'], format='%H:%M').dt.time

/tmp/ipykernel_5244/1302485406.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time'])


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Invoice ID               1000 non-null   object        
 1   Branch                   1000 non-null   object        
 2   City                     1000 non-null   object        
 3   Customer type            1000 non-null   object        
 4   Gender                   1000 non-null   object        
 5   Product line             1000 non-null   object        
 6   Unit price               1000 non-null   float64       
 7   Quantity                 1000 non-null   int64         
 8   Tax 5%                   1000 non-null   float64       
 9   Total                    1000 non-null   float64       
 10  Date                     1000 non-null   datetime64[ns]
 11  Time                     1000 non-null   datetime64[ns]
 12  Payment                  1000 non-n

In [91]:
df = df.drop_duplicates().reset_index(drop=True)
df.head(5)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,2024-01-26 13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,2024-01-26 10:29:00,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,2024-01-26 13:23:00,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27,2024-01-26 20:33:00,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,2024-01-26 10:37:00,Ewallet,604.17,4.761905,30.2085,5.3


In [98]:
class DataProcessor:
    def __init__(self, df):
        self.df = df

    def process_datetime_dim(self):
        datetime_dim = self.df[['Time', 'Date']].reset_index(drop=True)

        # Extract purchase_time in the desired format
        datetime_dim['purchase_time'] = datetime_dim['Time'].apply(lambda x: x.strftime('%I:%M %p') if pd.notna(x) else '')
        
        datetime_dim['datetime_id'] = datetime_dim.index

        # Split purchase_time into new columns
        split_time = datetime_dim['purchase_time'].str.split(':', expand=True)
        datetime_dim['purchase_time_hour'] = split_time[0]
        datetime_dim['purchase_time_minute'] = split_time[1].str.split().str[0]
        datetime_dim['purchase_time_AM/PM'] = split_time[1].str.split().str[1]

        # Extract other date-related features
        datetime_dim['sales_date'] = datetime_dim['Date']
        datetime_dim['sales_date_day'] = datetime_dim['Time'].dt.day_name()
        datetime_dim['sales_date_month'] = datetime_dim['Time'].dt.month_name()
        datetime_dim['sales_date_year'] = datetime_dim['Time'].dt.year
        datetime_dim['sales_date_weekday'] = datetime_dim['Time'].dt.weekday

        # Add a datetime_id column
        datetime_dim['datetime_id'] = datetime_dim.index

        # Reorder columns
        datetime_dim = datetime_dim[['datetime_id', 'purchase_time', 'purchase_time_hour', 'purchase_time_minute', 'purchase_time_AM/PM',
                                     'sales_date', 'sales_date_day', 'sales_date_month', 'sales_date_year', 'sales_date_weekday']]

        return datetime_dim['datetime_id']
    
    

    def process_payment_dim(self):
        payment_dim = self.df[['Payment']].reset_index(drop=True)
        payment_dim['payment_id'] = payment_dim.index
        label_encoder = LabelEncoder()
        payment_dim['payment_encoded'] = label_encoder.fit_transform(payment_dim['Payment'])
        payment_dim = payment_dim[['payment_id', 'payment_encoded', 'Payment']]
        return payment_dim['payment_id']
    
    def process_customer_dim(self):
        customer_dim = self.df[['Customer type', 'Gender', 'Rating']].reset_index(drop=True)
        customer_dim['customer_id'] = customer_dim.index
        label_encoder = LabelEncoder()
        customer_dim['customer_type_encoded'] = label_encoder.fit_transform(customer_dim['Customer type'])
        customer_dim['customer_gender_encoded'] = label_encoder.fit_transform(customer_dim['Gender'])
        bins = [1.0, 3.0, 5.0, 8.0, 10.0]
        labels = ['poor', 'good', 'very_good', 'excellent']
        customer_dim['rating_bin'] = pd.cut(customer_dim['Rating'], bins=bins, labels=labels)
        customer_dim = customer_dim[['customer_id', 'customer_type_encoded', 'customer_gender_encoded', 'rating_bin', 'Customer type']]
        return customer_dim['customer_id']
    
    

    def process_product_dim(self):
        product_dim = self.df[['Product line']].reset_index(drop=True)
        product_dim['product_id'] = product_dim.index
        label_encoder = LabelEncoder()
        product_dim['product_line_encoded'] = label_encoder.fit_transform(product_dim['Product line'])
        product_dim = product_dim[['product_id', 'Product line', 'product_line_encoded']]
        return product_dim['product_id']
    
    

    def process_super_center_dim(self):
        super_center_dim = self.df[['Branch', 'City']].reset_index(drop=True)
        super_center_dim['super_center_id'] = super_center_dim.index
        label_encoder = LabelEncoder()
        super_center_dim['branch_encoded'] = label_encoder.fit_transform(super_center_dim['Branch'])
        super_center_dim['branch_city_encoded'] = label_encoder.fit_transform(super_center_dim['City'])
        super_center_dim = super_center_dim[['Branch', 'City', 'super_center_id', 'branch_encoded', 'branch_city_encoded']]
        return super_center_dim['super_center_id']
    
    
    
    def process_fact_table(self):
        datetime_dim_result = self.process_datetime_dim()
        payment_dim_result = self.process_payment_dim()
        customer_dim_result = self.process_customer_dim()
        product_dim_result = self.process_product_dim()
        super_center_dim_result = self.process_super_center_dim()

    # Creating a fact table by combining dimensions and adding metrics
        fact_table = pd.merge(datetime_dim_result, payment_dim_result, how='inner', left_on='datetime_id', right_on='payment_id')
        fact_table = pd.merge(fact_table, customer_dim_result, how='inner', left_on='datetime_id', right_on='customer_id')
        fact_table = pd.merge(fact_table, product_dim_result, how='inner', left_on='datetime_id', right_on='product_id')
        fact_table = pd.merge(fact_table, super_center_dim_result, how='inner', left_on='datetime_id', right_on='super_center_id')

    # Appending additional columns to the fact table
        fact_table['total_cost'] = self.df['Total']
        fact_table['5%_tax'] = self.df['Tax 5%']
        fact_table['total_quantity'] = self.df['Quantity']
        fact_table['rating'] = self.df['Rating']
        fact_table['unit_price'] = self.df['Unit price']
        fact_table['cogs'] = self.df['cogs']
        fact_table['total_gross_margin_percent'] = self.df['gross margin percentage']
        fact_table['total_gross_income'] = self.df['gross income']

        return fact_table

processor = DataProcessor(df)
print(processor.process_fact_table())



     datetime_id  payment_id  customer_id  product_id  super_center_id  \
0              0           0            0           0                0   
1              1           1            1           1                1   
2              2           2            2           2                2   
3              3           3            3           3                3   
4              4           4            4           4                4   
..           ...         ...          ...         ...              ...   
995          995         995          995         995              995   
996          996         996          996         996              996   
997          997         997          997         997              997   
998          998         998          998         998              998   
999          999         999          999         999              999   

     total_cost   5%_tax  total_quantity  rating  unit_price    cogs  \
0      548.9715  26.1415               